In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Input
from tensorflow.keras.models import Model
from sklearn.metrics import r2_score

In [2]:
with open('data/board_states.npy', 'rb') as boards_file, \
    open('data/winners.npy', 'rb') as winners_file:
    boards = np.load(boards_file)
    shape = boards.shape
    boards = np.array([np.subtract(x[:,:,0], x[:,:,1]) for x in boards]).reshape((shape[0], shape[1], shape[2], 1))
    boards_3 = np.tile(boards,3)
    winners = np.load(winners_file)
    
    nums = np.arange(len(winners))
    np.random.shuffle(nums)
    X_test = boards_3[nums[:100000],:,:,:]
    y_test = winners[nums[:100000],:]

    print(boards_3.shape, winners.shape)
    print(X_test.shape, y_test.shape)

(1489941, 13, 13, 3) (1489941, 1)
(100000, 13, 13, 3) (100000, 1)


In [3]:
START_LEARNING_RATE = 0.0024787521766663585
NUM_EPOCHS = 10
BATCH_SIZE = 1000

def model_function():
    # a layer instance is callable on a tensor, and returns a tensor
    model = tf.keras.Sequential([
        Conv2D(16, kernel_size=(2,2), strides=(1,1),
                      data_format='channels_last',
                      activation='relu',
                      input_shape=(13, 13, 3)),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Conv2D(256, kernel_size=(3,3), strides=(1,1),
                      activation='relu'),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Flatten(),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.train.AdamOptimizer(START_LEARNING_RATE),
              loss=tf.keras.losses.binary_crossentropy,       # mean squared error
              metrics=['accuracy'])  # mean absolute error
    
    return model

In [ ]:
model = model_function()
model.fit(boards_3,winners, verbose=1, batch_size = BATCH_SIZE,
              epochs=NUM_EPOCHS)

Epoch 1/10
1489941/1489941 [==============================] - 184s 124us/step - loss: 0.5719 - acc: 0.6918
Epoch 2/10
1489941/1489941 [==============================] - 317s 212us/step - loss: 0.4840 - acc: 0.7557
Epoch 3/10
1489941/1489941 [==============================] - 303s 203us/step - loss: 0.4367 - acc: 0.7830
Epoch 4/10
1489941/1489941 [==============================] - 298s 200us/step - loss: 0.4088 - acc: 0.7977
Epoch 5/10
1323000/1489941 [=========================>....] - ETA: 33s - loss: 0.3914 - acc: 0.8068

In [11]:
model.save("./natalie_model_final")

In [12]:
model.save_weights("./natalie_model_final_weights")

In [13]:
evaluate = model.evaluate(X_test, y_test, batch_size = BATCH_SIZE)
print("Loss: ", evaluate[0])

100000/100000 [==============================] - 7s 65us/step
Loss:  0.106042401269


In [14]:
predict = model.predict(X_test)
print("R^2: ", r2_score(y_test, predict))

R^2:  0.575830364795


In [20]:
predict = model.predict(boards_3)

In [20]:
print("R^2: ", r2_score(winners, predict))

R^2:  0.513955074774


In [21]:
x=np.sum(np.abs(np.subtract(np.rint(predict),winners)))

In [22]:
1- x/len(winners)

0.84715770624474396